In [1]:
import pandas as pd

In [10]:
train = pd.read_csv('train_data.csv')
BB = pd.read_csv('broadband_data.csv')
server = pd.read_csv('server_data.csv')
outage = pd.read_csv('outage_data.csv')
rep = pd.read_csv('report_data.csv')
test = pd.read_csv('test_data.csv')
display(train.head())
display(BB.head())
display(server.head())
display(outage.head())
display(rep.head())
display(test.head())

,id,area_code,outage_duration
0,13366,area_415,1
1,6783,area_474,0
2,9519,area_931,1
3,10202,area_700,1
4,4555,area_600,2


,id,broadband_type
0,6597,broadband_type_8
1,8011,broadband_type_8
2,2597,broadband_type_8
3,5022,broadband_type_8
4,6852,broadband_type_8


,id,transit_server_type
0,6597,transit_server_type_11
1,8011,transit_server_type_15
2,2597,transit_server_type_15
3,5022,transit_server_type_15
4,5022,transit_server_type_11


,id,outage_type
0,6597,outage_type_2
1,8011,outage_type_2
2,2597,outage_type_2
3,5022,outage_type_1
4,6852,outage_type_1


,id,log_report_type,volume
0,6597,log_report_type_68,6
1,8011,log_report_type_68,7
2,2597,log_report_type_68,1
3,5022,log_report_type_172,2
4,5022,log_report_type_56,1


,id,area_code
0,3340,area_344
1,14067,area_933
2,1134,area_16
3,27,area_793
4,9793,area_344


In [21]:
#training dataset prep
temp = pd.merge(train,BB,on='id',how='left')
temp1 = pd.merge(temp,server,on='id',how='left')
temp2 = pd.merge(temp1,outage,on='id',how='left')
train_df = pd.merge(temp2,rep,on='id',how='left')
train_df.head()

,id,area_code,outage_duration,broadband_type,transit_server_type,outage_type,log_report_type,volume
0,13366,area_415,1,broadband_type_2,transit_server_type_35,outage_type_4,log_report_type_312,1
1,6783,area_474,0,broadband_type_2,transit_server_type_35,outage_type_2,log_report_type_312,2
2,6783,area_474,0,broadband_type_2,transit_server_type_35,outage_type_2,log_report_type_233,1
3,6783,area_474,0,broadband_type_2,transit_server_type_35,outage_type_2,log_report_type_232,1
4,6783,area_474,0,broadband_type_2,transit_server_type_35,outage_type_2,log_report_type_315,2


In [22]:
#test dataset prep
temp = pd.merge(test,BB,on='id',how='left')
temp1 = pd.merge(temp,server,on='id',how='left')
temp2 = pd.merge(temp1,outage,on='id',how='left')
test_df = pd.merge(temp2,rep,on='id',how='left')
test_df.head()

,id,area_code,broadband_type,transit_server_type,outage_type,log_report_type,volume
0,3340,area_344,broadband_type_2,transit_server_type_35,outage_type_4,log_report_type_312,2
1,14067,area_933,broadband_type_2,transit_server_type_20,outage_type_1,log_report_type_134,1
2,1134,area_16,broadband_type_8,transit_server_type_11,outage_type_1,log_report_type_155,10
3,1134,area_16,broadband_type_8,transit_server_type_11,outage_type_1,log_report_type_170,6
4,27,area_793,broadband_type_8,transit_server_type_11,outage_type_1,log_report_type_73,3


In [34]:
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [30]:
label_encoder = LabelEncoder() 
train_df['broadband_type']= label_encoder.fit_transform(train_df['broadband_type']) 
train_df['area_code']= label_encoder.fit_transform(train_df['area_code']) 
train_df['transit_server_type']= label_encoder.fit_transform(train_df['transit_server_type']) 
train_df['outage_type']= label_encoder.fit_transform(train_df['outage_type']) 
train_df['log_report_type']= label_encoder.fit_transform(train_df['log_report_type']) 

In [65]:
X = train_df.drop('outage_duration',axis=1)
y= train_df['outage_duration']

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [68]:
pred = model.predict(X_test)

In [69]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,pred)

0.7834195480533623

In [74]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier()
model.fit(X_train,y_train)
pred = model.predict(X_test)
accuracy_score(y_test,pred)

0.8289545330792268